In [91]:
import json
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import minmax_scale

from catsim.initialization import Initializer
from catsim.selection import MaxInfoSelector
from catsim.selection import UrrySelector
from catsim.irt import normalize_item_bank
from catsim.initialization import RandomInitializer
from catsim.simulation import Simulator
from catsim.stopping import MaxItemStopper
from catsim.estimation import NumericalSearchEstimator
from catsim.cat import generate_item_bank
from sklearn.feature_extraction.text import CountVectorizer

from girth import ability_eap, ability_map, ability_mle

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer 

from readability import Readability

In [9]:
with open('Data/data.json', 'r', encoding="utf-8") as handle:
    dataf = json.load(handle)

testConversation = list(dataf[3].keys())[0]
print(testConversation)

W: So, I've had a look at your résumé, and you seem to have the relevant work experience. I've also checked your references, and your former employers all have nothing but good things to say about you. ---M: That's good to hear. I've been very fortunate to have worked with some great companies, under strong and inspiring leadership. ---W: Well, I just have a couple of questions for you. Where do you see yourself five years from now? ---M: Well, at my age, I'm really looking for a job that can offer me some security. Most importantly, I want a job that can give me the opportunity to build a successful career. (12) (13) I don't want to put myself in a position where I am unable to reach my full potential. (13)---W: I understand. You're open to movement within the company? ---M: Exactly. In fact, the reason I left my last two jobs was because there was no room for promotion. (13)---W: You seem very determined. We like that. I'm certain you will be given every opportunity when the appropri

In [10]:
testQuestions = list(dataf[3].values())[0]
print(len(testQuestions))

3


In [11]:
# class Tokenizer(object):
#     def __call__(self, tweets):
#         return word_tokenize(tweets)

# class LemmaTokenizer(object):
    
#     def __init__(self):
#         self.wnl = WordNetLemmatizer()

#     def __call__(self, tweets):
#         return [self.wnl.lemmatize(self.wnl.lemmatize(self.wnl.lemmatize(tok, pos='n'), 
#                                                       pos='v'), pos='a') for tok in word_tokenize(tweets)]


In [58]:
def getMaterialDifficulty(dataf):
    
    materialsDifficultyLevel = []
    # questionsDifficultyLevel = []

    for data in tqdm(dataf):
        conv = list(data.keys())[0]
        conv = re.sub(r'\(*\d*\)', r'',conv)
        
        ques = list(data.values())[0]
        lenQues = len(ques)
    #     ques = list(data.values())[0]

        if len(conv.split()) < 100:
            materialsDifficultyLevel.append(1*0.5*lenQues)
        else:
            readability = Readability(conv)
            readabilityLevel = readability.dale_chall()
            materialsDifficultyLevel.append((readabilityLevel.score)*0.5*lenQues)   
    
#     materialDifficultyLevelNor = np.divide(materialsDifficultyLevel, max(materialsDifficultyLevel))
    materialDifficultyLevelNor = minmax_scale(materialsDifficultyLevel, feature_range=(-1, 1))
    
    return materialDifficultyLevelNor

def getQuestionsDifficulty(dataf, materialDifficultyLevelNor):
    questionsDifficultyLevelNor = []
    for i, data in tqdm(enumerate(dataf)):
        ques = list(data.values())[0]   
        questionsDifficultyLevelNor.append([list(materialDifficultyLevelNor[i])]*len(ques))
    
    return questionsDifficultyLevelNor

In [72]:
matDiff = getMaterialDifficulty(dataf=dataf)
quesDiff = getQuestionsDifficulty(dataf=dataf, materialDifficultyLevelNor=matDiff)
print(matDiff)
print(quesDiff)

100%|███████████████████████████████████████████████████████████████████████████| 12447/12447 [00:42<00:00, 293.18it/s]
0it [00:00, ?it/s]


TypeError: 'numpy.float64' object is not iterable

In [60]:
matDiff = pd.read_csv("Data/Diffculty Level/Mateials_diffculty_discrimination_by_cluster_without_quesLen.csv")
quesDiff = getQuestionsDifficulty(dataf=dataf, materialDifficultyLevelNor=matDiff.values)
display(matDiff)
matDiff = matDiff.values
print(matDiff)
print(quesDiff)

12447it [00:00, 417234.78it/s]


,discrimination,Materials Difficulty
0,1,-1.99
1,1,-0.99
2,2,2.01
3,3,2.02
4,3,-2.99
...,...,...
12442,1,-1.88
12443,2,0.12
12444,1,3.26
12445,1,1.48


[[ 1.   -1.99]
 [ 1.   -0.99]
 [ 2.    2.01]
 ...
 [ 1.    3.26]
 [ 1.    1.48]
 [ 1.   -1.87]]
[[[1.0, -1.99]], [[1.0, -0.99]], [[2.0, 2.01], [2.0, 2.01]], [[3.0, 2.02], [3.0, 2.02], [3.0, 2.02]], [[3.0, -2.99], [3.0, -2.99], [3.0, -2.99]], [[4.0, 0.01], [4.0, 0.01], [4.0, 0.01], [4.0, 0.01]], [[1.0, -1.98]], [[1.0, 1.01]], [[1.0, -0.98]], [[1.0, 1.02]], [[3.0, -2.98], [3.0, -2.98], [3.0, -2.98]], [[4.0, 0.02], [4.0, 0.02], [4.0, 0.02], [4.0, 0.02]], [[3.0, 0.03], [3.0, 0.03], [3.0, 0.03]], [[1.0, 3.01]], [[1.0, -1.97]], [[1.0, -1.96]], [[3.0, 2.03], [3.0, 2.03], [3.0, 2.03]], [[2.0, 2.04], [2.0, 2.04]], [[2.0, -2.97], [2.0, -2.97]], [[1.0, -0.97]], [[3.0, 0.04], [3.0, 0.04], [3.0, 0.04]], [[2.0, 0.05], [2.0, 0.05]], [[1.0, -1.95]], [[1.0, 3.02]], [[3.0, 0.06], [3.0, 0.06], [3.0, 0.06]], [[2.0, 0.07], [2.0, 0.07]], [[2.0, 2.05], [2.0, 2.05]], [[3.0, 2.06], [3.0, 2.06], [3.0, 2.06]], [[3.0, -2.96], [3.0, -2.96], [3.0, -2.96]], [[1.0, 1.03]], [[1.0, -1.94]], [[1.0, -1.93]], [[1.0, -0.96

In [61]:
materialDifficultyLevelNor = matDiff
questionsDifficultyLevelNor = quesDiff

In [21]:
# materialsDifficultyLevel = []
# # questionsDifficultyLevel = []

# for data in dataf:
#     conv = list(data.keys())[0]
#     conv = re.sub(r'\(*\d*\)', r'',conv)
    
#     ques = list(data.values())[0]
#     lenQues = len(ques)
# #     ques = list(data.values())[0]
    
#     if len(conv.split()) < 100:
#         materialsDifficultyLevel.append(1 + (0.5*lenQues))
#     else:
#         readability = Readability(conv)
#         readabilityLevel = readability.dale_chall()
#         materialsDifficultyLevel.append(readabilityLevel.score + (0.5*lenQues))   
    
# #     questionsDifficultLevel.append([readabilityLevel.score]*len(ques))
    
# print(materialsDifficultyLevel)

In [22]:
# materialsDifficultyLevel = []

# for data in dataf:
#     conv = list(data.keys())[0]
#     conv = re.sub(r'\(*\d*\)', r'',conv)
    
#     ques = list(data.values())[0]
#     lenQues = len(ques)
# #     ques = list(data.values())[0]
    
#     if len(conv.split()) < 100:
#         materialsDifficultyLevel.append(1 + (0.5*lenQues))
#     else:
#         readability = Readability(conv)
#         readabilityLevel = readability.flesch_kincaid()
#         materialsDifficultyLevel.append(readabilityLevel.score + (0.5*lenQues))   
    
# #     questionsDifficultLevel.append([readabilityLevel.score]*len(ques))
    
# print(materialsDifficultyLevel)

In [24]:
print("Max", max(materialsDifficultyLevel))
print("Min", min(materialsDifficultyLevel))
materialDifficultyLevelNor = np.divide(materialsDifficultyLevel, max(materialsDifficultyLevel))
print(materialDifficultyLevelNor)

NameError: name 'materialsDifficultyLevel' is not defined

In [8]:
questionsDifficultyLevelNor = []
for i, data in enumerate(dataf):
    ques = list(data.values())[0]   
    questionsDifficultyLevelNor.append([materialDifficultyLevelNor[i]]*len(ques))
print(questionsDifficultyLevelNor)

[[0.018017541814169827], [0.018017541814169827], [0.036035083628339654, 0.036035083628339654], [0.40911794199233475, 0.40911794199233475, 0.40911794199233475], [0.42921441102485397, 0.42921441102485397, 0.42921441102485397], [0.5857681378126486, 0.5857681378126486, 0.5857681378126486, 0.5857681378126486], [0.018017541814169827], [0.018017541814169827], [0.018017541814169827], [0.018017541814169827], [0.4386906749049334, 0.4386906749049334, 0.4386906749049334], [0.5266653965148899, 0.5266653965148899, 0.5266653965148899, 0.5266653965148899], [0.3864063310925368, 0.3864063310925368, 0.3864063310925368], [0.018017541814169827], [0.018017541814169827], [0.018017541814169827], [0.05405262544250948, 0.05405262544250948, 0.05405262544250948], [0.036035083628339654, 0.036035083628339654], [0.036035083628339654, 0.036035083628339654], [0.018017541814169827], [0.38064543148178664, 0.38064543148178664, 0.38064543148178664], [0.24703087919101552, 0.24703087919101552], [0.018017541814169827], [0.01

In [49]:
print(materialDifficultyLevelNor[2])
print(questionsDifficultyLevelNor[2])

[2.   2.01]
[array([2.  , 2.01]), array([2.  , 2.01])]


In [41]:
level1 = []
level2 = []
level3 = []
level4 = []
level5 = []
level6 = []
level7 = []
level8 = []

for index, score in enumerate(materialDifficultyLevelNor):
    if score < (5.0 / max(materialsDifficultyLevel)):
        level1.append(index)
    elif score < (6.0 / max(materialsDifficultyLevel)):
        level2.append(index)
    elif score < (7.0 / max(materialsDifficultyLevel)):
        level3.append(index)
    elif score < (8.0 / max(materialsDifficultyLevel)):
        level4.append(index)
    elif score < (9.0 / max(materialsDifficultyLevel)):
        level5.append(index)
    elif score < (10.0 / max(materialsDifficultyLevel)):
        level6.append(index)
    elif score < (11.0 / max(materialsDifficultyLevel)):
        level7.append(index)
    else:
        level8.append(index)

In [23]:
# print(level1)

In [89]:
administered_items = []

print(np.array(materialDifficultyLevelNor).shape)
print(np.array(questionsDifficultyLevelNor[0]).shape)
items = normalize_item_bank(np.array(materialDifficultyLevelNor))

initAbility = RandomInitializer().initialize()

selectedItems = UrrySelector().select(items=items, administered_items=administered_items, est_theta=initAbility)
administered_items.append(selectedItems)

print("Item parameter: ", items)
print("Initial ability: ", initAbility)
print("Selected Item: ", administered_items)
print("Slected Item materialDifficultLevel: ", materialDifficultyLevelNor[administered_items[0]])
print("Slected Item questionDifficultLevel: ", questionsDifficultyLevelNor[administered_items[0]])


(12447, 2)
(1, 2)
Item parameter:  [[ 1.   -1.99  0.    1.  ]
 [ 1.   -0.99  0.    1.  ]
 [ 2.    2.01  0.    1.  ]
 ...
 [ 1.    3.26  0.    1.  ]
 [ 1.    1.48  0.    1.  ]
 [ 1.   -1.87  0.    1.  ]]
Initial ability:  0.6050272058994306
Selected Item:  [9718]
Slected Item materialDifficultLevel:  [2.   0.49]
Slected Item questionDifficultLevel:  [[2.0, 0.4900000000000002], [2.0, 0.4900000000000002]]


In [90]:
item = np.array(questionsDifficultyLevelNor[administered_items[0]])
print(item)

items_parameter = normalize_item_bank(item)

difficulty = items_parameter[:,1]
print(difficulty)

discrimination = items_parameter[:,0]
print(discrimination)

respones = np.array([[False]*len(item)])
respones_list = np.array([[1] for i in range(len(item))])
# respones_list = [[False]*len(item[0])]
selected_items = np.array([i for i in range(len(item))])

print(items_parameter)
print(respones)
print(respones_list)
print(selected_items)

new_ability = ability_eap(dataset=respones_list, difficulty=difficulty, discrimination=discrimination)
print(new_ability[0])

# Estimetor = NumericalSearchEstimator()
# newAbility = Estimetor.estimate(items=items_parameter, administered_items=selected_items, 
#                                 response_vector=respones, est_theta=initAbility)

print("New Ability: ", new_ability[0])

[[2.   0.49]
 [2.   0.49]]
[0.49 0.49]
[2. 2.]
[[2.   0.49 0.   1.  ]
 [2.   0.49 0.   1.  ]]
[[False False]]
[[1]
 [1]]
[0 1]
1.1229625731035688
New Ability:  1.1229625731035688


In [79]:
selector = UrrySelector()
new_selected_item = selector.select(items=items, administered_items=administered_items, est_theta=new_ability)
administered_items.append(new_selected_item)

print("New selected Item: ", new_selected_item)
print("adminstered item: ", administered_items)
print("Slected Item materialDifficultLevel: ", materialDifficultyLevelNor[new_selected_item])
print("Slected Item questionDifficultLevel: ", questionsDifficultyLevelNor[new_selected_item])


New selected Item:  10118
adminstered item:  [7134, 10118]
Slected Item materialDifficultLevel:  [1. 1.]
Slected Item questionDifficultLevel:  [[1.0, 1.0]]


In [1]:
class Basic_IRT_Adptive_Recommendation():
    
    def __init__(self, numberOfMaterials, materialDifficultyLevelNor, questionsDifficultyLevelNor):
        self.administered_items  = []
        self.selected_items_once = []
        self.item_parameter = normalize_item_bank(np.array([materialDifficultyLevelNor]).T)
        self.ability = 0
        self.numberOfMaterials = numberOfMaterials
#         self.materialDifficultyLevelNor = materialDifficultyLevelNor
        self.questionsDifficultyLevelNor = questionsDifficultyLevelNor
        
        
    def normalize_item_parameter(self, one_item_array):
        normalize_item = normalize_item_bank(one_item_array.T)
        return normalize_item
        
    def initial_ability(self):
        init_ability = RandomInitializer().initialize()
        self.ability = init_ability
        return init_ability
    
    def ability_estimator(self, respones_vector):
        item = []
        for i in self.selected_items_once:
            item.extend(self.questionsDifficultyLevelNor[i])
            
        items = np.array([item])
        items_parameter = normalize_item_bank(items.T)
        respones = respones_vector
        selected_items = np.array([i for i in range(len(item))])

        Estimetor = NumericalSearchEstimator(method="dichotomous")
        newAbility = Estimetor.estimate(items=items_parameter, administered_items=selected_items, 
                                        response_vector=respones, est_theta=self.ability)   
        self.ability = newAbility
        return self.ability
    
    def item_selector(self):
        self.selected_items_once = []
        for _ in range(self.numberOfMaterials):
            selector = UrrySelector()
            new_selected_item = selector.select(items=self.item_parameter, administered_items=self.administered_items, 
                                                est_theta=self.ability)
            self.administered_items.append(new_selected_item)
            self.selected_items_once.append(new_selected_item)
        return self.selected_items_once

In [87]:
irt = Basic_IRT_Adptive_Recommendation(numberOfMaterials=2, materialDifficultyLevelNor=materialDifficultyLevelNor, 
                                       questionsDifficultyLevelNor=questionsDifficultyLevelNor)
init = irt.initial_ability()
print(init)

select_items = irt.item_selector()
print(select_items)

1.635280412095037
[10655, 9606]


In [88]:
respones = []
for i in select_items:
    respones.extend([False]*(len(questionsDifficultyLevelNor[i])))

respones = np.array(respones)
print(respones)

[False False False False False False]


In [89]:
new_ability = irt.ability_estimator(respones_vector=respones)

In [90]:
print(new_ability)

1.3115436121697337


In [61]:
a = [[1], [2], [3]]
b = [[1], [2], [3]]
a.extend(b)
print(np.array(a))

[[1]
 [2]
 [3]
 [1]
 [2]
 [3]]
